In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from fastai.tabular import *
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
import itertools

Unzip data

In [0]:
!unzip /content/drive/My\ Drive/kaggle/MovieTweeting/movietweetings.zip -d /content/drive/My\ Drive/kaggle/MovieTweeting/

Archive:  /content/drive/My Drive/kaggle/MovieTweeting/movietweetings.zip
replace /content/drive/My Drive/kaggle/MovieTweeting/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/My Drive/kaggle/MovieTweeting/ratings.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/My Drive/kaggle/MovieTweeting/users.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Create datasets names list, also prepare column names, we have to add them, because there are no column names presented in the database

In [0]:
PATH = Path('/content/drive/My Drive/kaggle/MovieTweeting')

In [4]:
movies = pd.read_csv(Path(PATH, 'movies.dat'), sep='::', names=['id', 'name', 'genre'])
movies.genre = movies.genre.str.replace('|', ',')
movies.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,id,name,genre
0,8,Edison Kinetoscopic Record of a Sneeze (1894),"Documentary,Short"
1,10,La sortie des usines Lumière (1895),"Documentary,Short"
2,12,The Arrival of a Train (1896),"Documentary,Short"
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,91,Le manoir du diable (1896),"Short,Horror"


In [5]:
ratings = pd.read_csv(Path(PATH, 'ratings.dat'), sep='::', names=['user_id', 'movie_id', 'rating', 'rating_timestamp'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [6]:
ratings.head()

,user_id,movie_id,rating,rating_timestamp
0,1,111161,10,1373234211
1,1,117060,7,1373415231
2,1,120755,6,1373424360
3,1,317919,6,1373495763
4,1,454876,10,1373621125


In [7]:
users = pd.read_csv(Path(PATH, 'users.dat'), sep='::', names=['user_id', 'twitter_id'])
users.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,twitter_id
0,1,397291295
1,2,40501255
2,3,417333257
3,4,138805259
4,5,2452094989


In [8]:
joined = pd.merge(left=movies.sort_values(by='id'), right=ratings.sort_values(by='movie_id'), left_on='id', right_on='movie_id', how='inner')
joined.head()

,id,name,genre,user_id,movie_id,rating,rating_timestamp
0,8,Edison Kinetoscopic Record of a Sneeze (1894),"Documentary,Short",45154,8,5,1396981211
1,10,La sortie des usines Lumière (1895),"Documentary,Short",54337,10,10,1412878553
2,12,The Arrival of a Train (1896),"Documentary,Short",43407,12,10,1439248579
3,25,The Oxford and Cambridge University Boat Race ...,NaN,34505,25,8,1488189899
4,91,Le manoir du diable (1896),"Short,Horror",10882,91,6,1385233195


Let`s take a closer look at our data

In [9]:
joined.describe(include='all')

,id,name,genre,user_id,movie_id,rating,rating_timestamp
count,8.151310e+05,815131,814694,815131.000000,8.151310e+05,815131.000000,8.151310e+05
unique,NaN,34377,2693,NaN,NaN,NaN,NaN
top,NaN,Gravity (2013),Drama,NaN,NaN,NaN,NaN
freq,NaN,3066,36719,NaN,NaN,NaN,NaN
mean,2.011177e+06,NaN,NaN,30130.203505,2.011177e+06,7.305341,1.447923e+09
std,1.736428e+06,NaN,NaN,17222.313958,1.736428e+06,1.855679,6.020674e+07
min,8.000000e+00,NaN,NaN,1.000000,8.000000e+00,0.000000,1.362062e+09
25%,7.654430e+05,NaN,NaN,15399.000000,7.654430e+05,6.000000,1.393059e+09
50%,1.675434e+06,NaN,NaN,30148.000000,1.675434e+06,7.000000,1.438336e+09
75%,2.660888e+06,NaN,NaN,44890.000000,2.660888e+06,9.000000,1.496029e+09


Check data for NaN values

In [10]:
for column in joined.columns:
  print('NaN in ', column, ' = ', joined[column].isna().sum())

NaN in  id  =  0
NaN in  name  =  0
NaN in  genre  =  437
NaN in  user_id  =  0
NaN in  movie_id  =  0
NaN in  rating  =  0
NaN in  rating_timestamp  =  0


As there is very few nan comparing to overall dataset size, we can simply drop them

In [0]:
joined = joined.dropna(axis=0)

Get dummies for genre column

In [0]:
joined = pd.concat([joined.drop('genre', 1), joined.genre.str.get_dummies(sep=',')], axis=1)

In [0]:
joined.head()

In [0]:
joined.to_pickle(Path(PATH/'movies_preprocessed.pkl'))